Wir implementieren den Perzeptron-Algorithmus, und wenden den an den Iris-Datensatz, um eine binäre Klassifikation zu bekommen, damit wir die Blumen zwischen Klassen "Iris-setosa" und "Iris- versicolor" bzw. "Iris-setosa" und "Iris-virginica" unterscheiden.

In [1]:
import numpy as np

def file_get_contents(filename):
    return np.genfromtxt(filename, delimiter=',')

Daten und Klassen vorbereiten.

In [2]:
originaldata = file_get_contents('resources/iris.data')
setosa = originaldata[ :50]
versicolor = originaldata[50: 100]
virginica = originaldata[100: ]

Nehmen wir 20% zufälligen Dateien als Testdaten, und die restliche als Trainingdaten.

In [3]:
def get_testdata(data):
    length = int(0.2 * len(data))
    return data[:length]


def get_trainingdata(data):
    length = int(0.8 * len(data))
    return data[:length]


def get_initial(data1, data2):
    for i in range(0, len(data1)):
            data1[i][len(data1[i])-1] = 1

    for j in range(0, len(data2)):
        data2[j][len(data2[j]) - 1] = -1

    data = np.concatenate((data1, data2), axis=0)
    np.random.shuffle(data)
    return data

test = get_testdata(get_initial(setosa, versicolor))
training = get_trainingdata(get_initial(setosa, versicolor))

In [4]:
def initial_weight(x):

    length = len(x)-1
    y = x[length]

    w = np.random.randint(10, size=(length, 1))
    tmp = np.dot(y, np.transpose(w))

    return np.dot(tmp, np.array(x[:-1]))

hier berechnen wir Gewicht Vektor w und bias b, zu ersr wird w und b zufällig initialisiert, danach wird es immer 
aktualisiert jedes Mal wenn es einen falsch gelabled sample gibt, es wird so lange wiederholt bis alle Samples richtig gelabled sind. 

Mit Perzeptron Algorithmus wird die alle falsch gelabeled Datensätze "extrem" mehr gelernt, durch die Veränderung der demgegende Gewichte, die schon richtig gelabeled Datensätze können wir schon relativ leicht ignorieren.
Wir wiederholen die "adjusting" Phase so lange, bis alle Datensätze richtig klassifiziert werden. 
(Nur wenn die Daten linear separierbar sind, ansonsten müssen wir mit einer festlegten Iterationsanzahl lernenn)

In [5]:
def all_checked(w, b, training):
    for i in range(0, len(training)):
        x = training[i]
        if (x[-1] == 1.0):
            if ((np.dot(w, x) + b) < 0):
                return False
        elif (np.dot(w, x) >= 0):
            return False
    return True


def update(w, b ,x):
    y = x[-1]
    w = np.add(w, 0.95*(np.dot(y, x)))
    b = b + 0.95*y
    return w, b


def get_weight(training):
    length = len(training)
    column = np.ones((len(training), 1))
    training = np.concatenate((column, training), axis=1)

    w = np.transpose(np.random.randint(10, size=(len(training[0]-1), 1)))
    b = 0

    while(all_checked(w, b, training) == False): #sobald es noch falsche gelabled Sample gibt, widerholt es

        i = np.random.randint(0, length)
        x = training[i]
        if (x[-1] == 1.0 and (np.dot(w, x) + b) < 0): # Produkt mit w negativ aber positiv getagged Fall

            w = update(w, b, x)[0]
            b = update(w, b, x)[1]
        if (x[-1] == -1.0 and (np.dot(w, x) + b) > 0):
        # mit anderem Fall wandeln wir es zu erst ins positive Sample, damit wir die selbe update Funktion benutzen können
            x = np.dot(-1, x)

            w = update(w, b, x)[0]
            b = update(w, b, x)[1]
        if ((np.dot(w, x) + b) == 0):
            pass
    return w, b

 mit update Funktion wird das Gewicht Vektor und bias immer aktualisiert, w = w + ηy(i)x(i), b = b + ηy(i)
0.95 entspricht hier die learning rate, welches im Interval (0, 1) liegt.

Definieren wir jetzt eine Error Funktion, welche die Error Rate berechnet, und schauen wir uns mal das Lernergebnis an.

In [8]:
def error_rate(data1, data2): # kriegen Error rate mit linear separierbaren Datensätzen
    error = 0
    data = get_initial(data1, data2)
    training = get_trainingdata(data)
    column = np.ones((len(get_testdata(data)), 1))
    test = np.concatenate((column, get_testdata(data)), axis=1)

    weight = get_weight(training)[0]
    bias = get_weight(training)[1]

    for i in range(0, len(test)):
        if ((np.dot(weight, test[i]) + bias) >= 0 and (test[i][-1] == -1.0)):
            error += 1
        elif (np.dot(weight, test[i])+ bias) < 0 and (test[i][-1] == 1.0):
            error += 1
    return (error/len(test))*100


print('error rate between Iris-setosa and Iris-virginica is', error_rate(setosa, virginica), '%')
print('error rate between Iris-setosa and Iris-versicolor is', error_rate(setosa, versicolor), '%')


error rate between Iris-setosa and Iris-virginica is 0.0 %
error rate between Iris-setosa and Iris-versicolor is 0.0 %
